# Environement set up details

## Installing locally (rather than colab)

Using `uv` (from https://docs.astral.sh/uv/)

```
# install uv in its own global location (using pipx)
pipx install uv
# create a virtual environment
uv venv
# activate the environment
source .venv/bin/activate
# install the Jupyter notebook packages
uv pip install ipykernel jupyter notebook
# install required packages
uv pip install networkx matplotlib pandas mazelib imageio

In [ ]:
from typing import Tuple, List, Dict, Optional, Union, Callable, Set, Any
from dataclasses import dataclass, field, asdict
from abc import ABC, abstractmethod
import time
from datetime import datetime
from pathlib import Path
import json
from collections import deque

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import networkx as nx

from mazelib import Maze
from mazelib.generate.Sidewinder import Sidewinder
from mazelib.solve.BacktrackingSolver import BacktrackingSolver

# for creating the gif of the steps
import imageio
import matplotlib.animation as animation
import os
import tempfile

## Config

In [ ]:
@dataclass
class Config:
    """Configuration parameters for maze generation and search algorithms.

    This class centralizes all configuration options for maze environments and search
    algorithms to ensure consistent parameter usage throughout the system.

    Attributes:
        maze_size (int): Grid dimensions (nxn) of the maze. Default is 5x5.
        maze_id (Optional[int]): Seed for reproducible maze generation. If None, a random seed is used.
        visualization_delay (float): Delay in seconds between search algorithm steps for visualization. Default is 0.5.
        show_exploration (bool): Whether to visualize the exploration process. Default is True.
        max_steps (Optional[int]): Maximum steps for search algorithm execution before termination.
                                  None means unlimited steps allowed.
    """
    # Maze parameters
    maze_size: int = 5  # Grid dimensions (nxn)
    maze_id: Optional[int] = None  # Seed for reproducible maze generation

    # Visualization parameters
    visualization_delay: float = 0.5  # Delay between search steps
    show_exploration: bool = True  # Whether to visualize exploration

    # Search parameters
    max_steps: Optional[int] = None  # Maximum steps for search (None for unlimited)

## MazeEnvironment

In [ ]:
class MazeEnvironment:
    """Handles maze generation, state management and visualization.

    This class encapsulates all maze-related functionality, including generating mazes,
    managing maze state, creating graph representations for search algorithms, and
    providing visualization capabilities.

    Attributes:
        config (Config): Configuration parameters for the maze.
        grid (numpy.ndarray): Binary maze representation (0=path, 1=wall).
        start (Tuple[int, int]): Starting position, typically (1,1).
        end (Tuple[int, int]): Goal position, typically at the bottom-right corner.
        optimal_path (List[Tuple[int, int]]): Shortest solution path from start to end.
        optimal_path_length (int): Length of the shortest solution path.
        graph (Dict): Graph representation of maze for search algorithms.
    """

    def __init__(self, config: Config):
        """Initialize the maze environment with given configuration.

        Args:
            config: Configuration parameters for maze generation and visualization.
        """
        self.config = config
        self.grid = None
        self.start = None
        self.end = None
        self._maze = None
        self.seed = None
        self.optimal_path = None
        self.optimal_path_length = None
        self.graph = None
        self.generate()

    def generate(self) -> None:
        """Creates new maze using Sidewinder algorithm.

        Generates a random maze based on configuration parameters, establishes
        start and end positions, calculates optimal path, and creates graph
        representation for search algorithms.
        """
        # Use config maze_id if provided, otherwise generate random seed
        self.seed = self.config.maze_id if self.config.maze_id is not None else np.random.randint(1, 1000)

        self._maze = Maze(self.seed)
        self._maze.generator = Sidewinder(self.config.maze_size, self.config.maze_size)
        self._maze.generate()
        self._maze.generate_entrances()

        self.grid = self._maze.grid

        # Set the start to the first valid cell inside grid
        self.start = (1, 1)
        # Set the end to the last valid cell inside grid
        self.end = (self.grid.shape[0]-2, self.grid.shape[1]-2)

        # After maze generation, calculate optimal path
        self._calculate_optimal_path()

        # Create graph representation for search algorithms
        self._create_graph()

    def _calculate_optimal_path(self) -> None:
        """Calculate optimal path using maze's solver.

        Uses the BacktrackingSolver to find the optimal solution path from
        start to end. Sets optimal_path and optimal_path_length attributes.
        """
        # Set up solver
        self._maze.solver = BacktrackingSolver()
        self._maze.start = self.start
        self._maze.end = self.end

        # Solve
        self._maze.solve()

        if self._maze.solutions:
            self.optimal_path = self._maze.solutions[0]  # Store first solution
            self.optimal_path_length = len(self.optimal_path) + 1
        else:
            # Handle case where no solution is found
            self.optimal_path = None
            self.optimal_path_length = None

    def _create_graph(self) -> None:
        """Creates graph representation for search algorithms.

        Transforms the grid-based maze into an adjacency list graph representation
        where each non-wall cell is a node, and edges connect to adjacent non-wall cells.
        """
        self.graph = {}
        rows, cols = self.grid.shape

        # Define possible moves: up, right, down, left
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]

        for r in range(rows):
            for c in range(cols):
                # Skip walls
                if self.grid[r][c] == 1:
                    continue

                node = (r, c)
                neighbors = []

                # Check all four directions
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    # Check if the neighbor is valid
                    if self.is_valid_move((nr, nc)):
                        neighbors.append((nr, nc))

                self.graph[node] = neighbors

    def get_minimum_steps(self) -> Optional[int]:
        """Returns the length of optimal path if it exists.

        Returns:
            The number of steps in the optimal path from start to end,
            or None if no path exists.
        """
        return self.optimal_path_length

    def is_valid_move(self, state: Tuple[int, int]) -> bool:
        """Checks if a move to the given state is legal.

        Args:
            state: Coordinates (row, col) to check for validity.

        Returns:
            True if the move is valid (within bounds and not a wall),
            False otherwise.
        """
        row, col = state
        # check if the move goes outside of the grid or into a wall (1)
        if (row < 0 or row >= self.grid.shape[0] or
            col < 0 or col >= self.grid.shape[1] or
            self.grid[state] == 1):
            return False
        # move is valid
        return True

    def visualize(self, path: Optional[List[Tuple[int, int]]] = None,
                  visited: Optional[Set[Tuple[int, int]]] = None,
                  show_optimal: bool = False,
                  save_path: Optional[Path] = None,
                  title: Optional[str] = None) -> None:
        """Displays or saves maze visualization with optional path and visited nodes.

        Creates a color-coded visualization of the maze showing walls, paths,
        visited nodes, and solution paths.

        Args:
            path: Optional list of positions showing a solution path.
            visited: Optional set of visited positions during search.
            show_optimal: Whether to display the optimal path.
            save_path: If provided, saves figure to this path instead of displaying.
            title: Optional title for the plot.
        """
        plt.figure(figsize=(8, 8))

        # Add title showing Maze ID and minimum steps
        if title:
            plt.title(title)
        else:
            plt.title(f"Maze #{self.seed} - Min Steps: {self.get_minimum_steps()}")

        # Create a visualization grid filled with appropriate values for coloring
        rows, cols = self.grid.shape
        viz_grid = np.ones((rows, cols)) * 5  # Initialize with unvisited path value

        # Fill in walls
        viz_grid[self.grid == 1] = 0  # Walls

        # Fill in visited paths
        if visited:
            for pos in visited:
                r, c = pos
                if self.grid[r, c] == 0:  # Only if it's a path
                    viz_grid[r, c] = 1  # Visited paths

        # Fill in final path
        if path:
            for pos in path:
                if pos != self.start and pos != self.end:
                    r, c = pos
                    viz_grid[r, c] = 2  # Final path

        # Mark start and end
        viz_grid[self.start] = 3  # Start
        viz_grid[self.end] = 4    # Goal

        # Define colors: Wall, Visited, Final Path, Start, Goal, Unvisited
        cmap = plt.cm.colors.ListedColormap(['black', 'yellow', 'green', 'blue', 'purple', 'white'])
        bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
        norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

        # Plot the maze
        plt.imshow(viz_grid, cmap=cmap, norm=norm)
        plt.xticks([])
        plt.yticks([])

        # Add legend
        legend_elements = [
            plt.Rectangle((0,0), 1, 1, color='white', label='Path'),
            plt.Rectangle((0,0), 1, 1, color='yellow', label='Visited'),
            plt.Rectangle((0,0), 1, 1, color='black', label='Wall'),
            plt.Rectangle((0,0), 1, 1, color='green', label='Final Path'),
            plt.Rectangle((0,0), 1, 1, color='blue', label='Start'),
            plt.Rectangle((0,0), 1, 1, color='purple', label='Goal')
        ]
        plt.legend(handles=legend_elements, loc='upper center',
                  bbox_to_anchor=(0.5, -0.05), ncol=3)

        if save_path:
            plt.savefig(save_path, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

## SearchResult

In [ ]:
@dataclass
class SearchResult:
    """Enhanced container for search algorithm results with educational metrics.

    This class stores and analyzes the results of search algorithm execution, providing
    not only basic path information but also educational metrics and insights about
    algorithm performance. It includes capabilities for generating reports and
    visualizations of search results.

    Attributes:
        path (List[Tuple[int, int]]): Solution path from start to goal, if found.
        visited (List[Tuple[int, int]]): List of nodes visited during search, in order.
        success (bool): Whether the search found a valid path to the goal.
        steps (int): Number of algorithm steps executed during search.
        execution_time (float): Time taken for the search execution in seconds.
        exploration_history (List): History of algorithm state for visualization/analysis.
        node_discovery (Dict): Maps each node to the step when it was first discovered.
        node_expansion (Dict): Maps each node to the step when it was expanded.
    """
    path: Optional[List[Tuple[int, int]]] = None  # Solution path from start to goal
    visited: List[Tuple[int, int]] = field(default_factory=list)  # List of visited nodes
    success: bool = False  # Whether the search found a path
    steps: int = 0  # Number of algorithm steps executed
    execution_time: float = 0.0  # Time taken for the search
    exploration_history: List = field(default_factory=list)  # History of algorithm state (for visualization)

    # Educational tracking metrics
    node_discovery: Dict[Tuple[int, int], int] = field(default_factory=dict)  # When each node was discovered
    node_expansion: Dict[Tuple[int, int], int] = field(default_factory=dict)  # When each node was expanded

    def __str__(self) -> str:
        """Enhanced string representation of results with educational metrics.

        Returns:
            A formatted string with key performance metrics and educational information.
        """
        if self.success:
            return self._format_success_output()
        else:
            return self._format_failure_output()

    def _format_success_output(self) -> str:
        """Format output for successful search with educational information.

        Returns:
            A formatted string with performance metrics for successful searches.
        """
        avg_branching = self._calculate_avg_branching_factor()

        output = [
            f"✅ Search succeeded in {self.steps} steps ({self.execution_time:.3f}s)",
            f"📏 Path length: {len(self.path)} nodes",
            f"🔍 Visited nodes: {len(self.visited)} nodes ({(len(self.visited) / self.steps):.2f} nodes/step)",
            f"⚙️ Efficiency: {(len(self.path) / len(self.visited) * 100):.1f}% (path nodes / visited nodes)",
            f"🌲 Average branching factor: {avg_branching:.2f} neighbors/node",
            f"⏱️ Average time per step: {(self.execution_time / self.steps * 1000):.2f} ms"
        ]

        return "\n".join(output)

    def _format_failure_output(self) -> str:
        """Format output for failed search with educational information.

        Returns:
            A formatted string with performance metrics and possible failure reasons.
        """
        avg_branching = self._calculate_avg_branching_factor()

        output = [
            f"❌ Search failed after {self.steps} steps ({self.execution_time:.3f}s)",
            f"🔍 Visited nodes: {len(self.visited)} nodes ({(len(self.visited) / self.steps):.2f} nodes/step)",
            f"🌲 Average branching factor: {avg_branching:.2f} neighbors/node",
            f"⏱️ Average time per step: {(self.execution_time / self.steps * 1000):.2f} ms",
            f"💡 Possible reasons for failure:",
            f"   - No valid path exists between start and goal",
            f"   - Search exceeded maximum step limit ({self.steps} steps)",
            f"   - Maze structure prevents reaching the goal"
        ]

        return "\n".join(output)

    def _calculate_avg_branching_factor(self) -> float:
        """Calculate the average branching factor during search.

        Returns:
            The average number of new nodes discovered per expanded node,
            which approximates the branching factor of the search space.
        """
        if not self.node_expansion or len(self.node_expansion) <= 1:
            return 0.0

        # We can determine this from the ratio of discovered nodes to expanded nodes
        # Excluding the start node which doesn't have a parent
        discovered_count = len(self.node_discovery) - 1  # -1 for start node
        expanded_count = len(self.node_expansion)

        return discovered_count / expanded_count if expanded_count > 0 else 0.0

    def to_dict(self) -> Dict[str, Any]:
        """Convert results to dictionary for analysis with educational metrics.

        Returns:
            Dictionary containing all metrics and performance data for analysis.
        """
        return {
            'success': self.success,
            'steps': self.steps,
            'execution_time': self.execution_time,
            'path_length': len(self.path) if self.path else None,
            'visited_nodes': len(self.visited),
            'path_to_visited_ratio': (len(self.path) / len(self.visited)
                                     if self.path and len(self.visited) > 0 else None),
            'avg_branching_factor': self._calculate_avg_branching_factor(),
            'discovery_to_expansion_ratio': (len(self.node_discovery) / len(self.node_expansion)
                                           if self.node_expansion else None),
            'nodes_per_step': len(self.visited) / self.steps if self.steps > 0 else 0
        }


## Base Class - SearchAlgorithmBase

In [ ]:
class SearchAlgorithmBase(ABC):
    """Abstract base class for search algorithms.

    This class provides a common interface and shared functionality for all search
    algorithms. It handles common tasks such as error handling, timing, and basic
    visualization capabilities. Specific search algorithms should inherit from this
    class and implement the abstract search method.

    Attributes:
        env (MazeEnvironment): Reference to maze environment being searched.
        config (Config): Configuration parameters for the search algorithm.
        name (str): Algorithm name, derived from class name.
    """

    def __init__(self, env: MazeEnvironment):
        """Initialize the search algorithm with an environment.

        Args:
            env: The maze environment to search within.
        """
        self.env = env
        self.config = env.config
        self.name = self.__class__.__name__

    @abstractmethod
    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Search for a path from start to goal.

        This abstract method must be implemented by all concrete search algorithm classes.

        Args:
            start: The starting position coordinates (row, col).
            goal: The goal position coordinates (row, col).

        Returns:
            A SearchResult containing path, visited nodes, and performance metrics.
        """
        pass

    def run(self, start: Optional[Tuple[int, int]] = None,
            goal: Optional[Tuple[int, int]] = None) -> SearchResult:
        """Run the search algorithm with timing and error handling.

        Provides a consistent interface for running any search algorithm with
        proper timing measurement and error handling.

        Args:
            start: The starting position (defaults to environment start if None).
            goal: The goal position (defaults to environment end if None).

        Returns:
            A SearchResult object containing the search results and performance metrics.
        """
        # Use environment start/goal if not specified
        start = start if start is not None else self.env.start
        goal = goal if goal is not None else self.env.end

        start_time = time.time()
        try:
            result = self.search(start, goal)
        except Exception as e:
            print(f"Error in {self.name}: {str(e)}")
            result = SearchResult(success=False)

        result.execution_time = time.time() - start_time
        return result

    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Visualize the search process step by step.

        Creates an animated visualization of the search algorithm's execution
        by stepping through the exploration history.

        Args:
            result: The SearchResult from running the algorithm.
            delay: Time delay between steps in seconds (defaults to config delay if None).
        """
        if not result.exploration_history:
            print(f"No exploration history available for {self.name}")
            return

        print(f"The path is {result.path}")

        delay = delay if delay is not None else self.config.visualization_delay

        for i, state in enumerate(result.exploration_history):
            # Clear previous output
            clear_output(wait=True)

            # Customize this part based on how exploration_history is structured
            # This example assumes each state is a tuple (visited, frontier, current_path)
            visited, frontier, current_path = state

            # Display current search state
            if i == len(result.exploration_history) - 1 and result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
                current_path=result.path
            else:
                title = f"{self.name} - Step {i+1}/{len(result.exploration_history)}"

            self.env.visualize(path=current_path, visited=visited, title=title)

            # Add additional information
            print(f"Current frontier size: {len(frontier)}")
            print(f"Visited nodes: {len(visited)}")
            print(f"Current path length: {len(current_path) if current_path else 0}")

            plt.pause(delay)

        # If no exploration history was saved, show final state
        if not self.config.show_exploration:
            clear_output(wait=True)
            if result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
            else:
                title = f"{self.name} - No path found. ({result.steps} steps, {result.execution_time:.3f}s)"

            self.env.visualize(path=result.path, visited=set(result.visited), title=title)

        print(result)

## BreadthFirstSearch

In [ ]:
class BreadthFirstSearch(SearchAlgorithmBase):
    """Breadth-First Search implementation with enhanced educational output.

    This class implements the BFS algorithm for maze solving with detailed tracking
    of algorithm execution for educational purposes. BFS guarantees the shortest
    path in unweighted graphs by exploring nodes in order of their distance from
    the start node using a FIFO queue.

    The implementation provides comprehensive visualization and educational
    features to illustrate how BFS works step-by-step.
    """

    def _reconstruct_path(self, parent, start, goal):
        """Reconstruct the solution path from parent pointers.

        Args:
            parent: Dictionary mapping each node to its parent in the search tree.
            start: Starting position coordinates.
            goal: Goal position coordinates.

        Returns:
            List of coordinates representing the path from start to goal.
        """
        path = [goal]
        current = goal
        while current != start:
            current = parent[current]
            path.append(current)
        return path[::-1]  # Reverse to get start→goal

    def search(self, start: Tuple[int, int], goal: Tuple[int, int]) -> SearchResult:
        """Perform BFS search from start to goal with enhanced educational tracking.

        Implements the classic BFS algorithm using a queue data structure, with
        added tracking of educational metrics such as node discovery and expansion times.

        Args:
            start: Starting position coordinates (row, col).
            goal: Goal position coordinates (row, col).

        Returns:
            SearchResult object containing path, visited nodes, and educational metrics.
        """
        # Initialize the queue with the start node
        queue = deque([start])

        # Keep track of visited nodes to avoid cycles
        visited = set([start])
        visited_order = [start]
        parent = {start: None}

        # For visualization purposes
        exploration_history = []

        # Add educational tracking
        node_discovery = {start: 0}  # Maps node -> step when it was discovered
        node_expansion = {}          # Maps node -> step when it was expanded

        steps = 0
        max_steps = self.config.max_steps or float('inf')

        while queue and steps < max_steps:
            steps += 1

            # Dequeue the next node
            current_node = queue.popleft()
            node_expansion[current_node] = steps

            # Check if we've reached the goal
            if current_node == goal:
                # Reconstruct path when goal is found
                final_path = self._reconstruct_path(parent, start, goal)
                result = SearchResult(
                    path=final_path,
                    visited=visited_order,
                    success=True,
                    steps=steps,
                    exploration_history=exploration_history
                )
                # Add educational data to the result
                result.node_discovery = node_discovery
                result.node_expansion = node_expansion
                return result

            # Add all unvisited neighbors to the queue
            neighbors_added = []
            for neighbor in self.env.graph.get(current_node, []):
                if neighbor not in visited:
                    visited.add(neighbor)
                    visited_order.append(neighbor)
                    parent[neighbor] = current_node
                    queue.append(neighbor)
                    node_discovery[neighbor] = steps
                    neighbors_added.append(neighbor)

            if self.config.show_exploration:
                # Get current partial path for visualization
                current_partial_path = self._reconstruct_path(parent, start, current_node) if current_node != start else [start]

                # Educational information for this step
                step_info = {
                    "step": steps,
                    "expanded_node": current_node,
                    "neighbors_added": neighbors_added,
                    "queue_before": list(queue) if not neighbors_added else [n for n in list(queue) if n not in neighbors_added],
                    "queue_after": list(queue)
                }

                exploration_history.append((
                    visited.copy(),           # All visited nodes
                    list(queue),              # Current frontier
                    current_partial_path,     # Path to current node
                    step_info                 # Educational info
                ))

        # If we get here, there's no path to the goal
        result = SearchResult(
            path=None,
            visited=visited_order,
            success=False,
            steps=steps,
            exploration_history=exploration_history
        )
        result.node_discovery = node_discovery
        result.node_expansion = node_expansion
        return result

    def visualize_search(self, result: SearchResult, delay: float = None) -> None:
        """Enhanced visualization with educational commentary.

        Creates an animated step-by-step visualization of the BFS algorithm's
        execution with detailed educational explanations about each step.

        Args:
            result: The SearchResult from running the algorithm.
            delay: Time delay between visualization steps (defaults to config).
        """
        if not result.exploration_history:
            print(f"No exploration history available for {self.name}")
            return

        delay = delay if delay is not None else self.config.visualization_delay

        for i, state in enumerate(result.exploration_history):
            # Clear previous output
            clear_output(wait=True)

            # Enhanced exploration history structure
            visited, frontier, current_path, step_info = state

            # Format node coordinates nicely
            def format_node(node):
                return f"({node[0]},{node[1]})"

            # Educational commentary based on the step info
            expanded_node = format_node(step_info["expanded_node"])
            commentary = [
                f"Step {step_info['step']}: Expanding node {expanded_node}",
            ]

            # Add information about neighbors discovered
            if step_info["neighbors_added"]:
                neighbors_text = ", ".join([format_node(n) for n in step_info["neighbors_added"]])
                commentary.append(f"Discovered neighbors: {neighbors_text}")
                commentary.append(f"These neighbors were added to the queue because they haven't been visited before.")
            else:
                commentary.append(f"No new neighbors were discovered (all were already visited).")

            # Queue state before and after
            queue_before = ", ".join([format_node(n) for n in step_info["queue_before"]])
            queue_after = ", ".join([format_node(n) for n in step_info["queue_after"]])
            commentary.append(f"Queue before: [{queue_before}]")
            commentary.append(f"Queue after: [{queue_after}]")

            # Display title based on search progress
            if i == len(result.exploration_history) - 1 and result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
                current_path = result.path
            else:
                title = f"{self.name} - Step {i+1}/{len(result.exploration_history)}"

            # Display the maze visualization
            self.env.visualize(path=current_path, visited=visited, title=title)

            # Display educational commentary
            print("\n".join(commentary))

            # Additional stats
            print(f"\nVisited nodes: {len(visited)}")
            print(f"Queue size: {len(frontier)}")
            print(f"Current path length: {len(current_path) if current_path else 0}")

            plt.pause(delay)

        # Show final state if exploration wasn't saved
        if not self.config.show_exploration:
            clear_output(wait=True)
            if result.success:
                title = f"{self.name} - Path found! ({result.steps} steps, {result.execution_time:.3f}s)"
            else:
                title = f"{self.name} - No path found. ({result.steps} steps, {result.execution_time:.3f}s)"

            self.env.visualize(path=result.path, visited=set(result.visited), title=title)

        print(result)

## BFSEducationalDashboard

In [ ]:
class BFSEducationalDashboard:
    """Interactive dashboard for teaching BFS algorithm concepts.

    This class provides a comprehensive educational tool for visualizing and
    understanding the Breadth-First Search algorithm. It includes capabilities
    for step-by-step visualization, animated GIF creation, graph visualization,
    and interactive explanations of BFS concepts.

    The dashboard is designed for educational purposes, helping students and
    learners understand how BFS works by visualizing its execution in both
    maze grid and graph representations.

    Attributes:
        env (MazeEnvironment): The maze environment being searched.
        result (SearchResult): The results from running the BFS algorithm.
        steps_data (List): Processed exploration history data for visualization.
    """

    def __init__(self, env: MazeEnvironment, bfs_result: SearchResult):
        """Initialize the educational dashboard with search results.

        Args:
            env: The maze environment being searched.
            bfs_result: Results from running the BFS algorithm.
        """
        self.env = env
        self.result = bfs_result
        # Extract data from exploration history
        self._extract_history_data()

    def animate_bfs_on_graph(self, output_file='bfs_graph.gif', fps=1, size=5):
        """Animate BFS algorithm on graph representation using consistent drawing approaches.

        Creates a GIF animation showing how BFS explores the maze when represented
        as a graph, illustrating concepts like frontier, visited nodes, and path formation.

        Parameters:
            output_file: Path to save the output GIF.
            fps: Frames per second for the animation.
            size: Size of the output figure in inches.
        """
        try:
            # Create the graph representation of the maze
            G = self.create_maze_graph()

            # Create position mapping for nodes - define once and reuse
            pos = {node: (node[1], -node[0]) for node in G.nodes()}

            # Set consistent node sizes and styles
            node_size = 300
            edge_width = 1
            path_edge_width = 3

            # Set consistent colors
            colors = {
                'regular': 'lightgray',
                'visited': 'yellow',
                'queue': 'lightblue',
                'path': 'green',
                'start': 'blue',
                'end': 'purple',
                'expanded': 'red'
            }

            with tempfile.TemporaryDirectory() as temp_dir:
                frame_files = []

                # Create frames for each step
                for i, step_data in enumerate(self.steps_data):
                    try:
                        plt.figure(figsize=(size, size))

                        # Draw base graph elements
                        # 1. Regular nodes (all nodes as background)
                        nx.draw_networkx_nodes(G, pos, node_color=colors['regular'], node_size=node_size)

                        # 2. Visited nodes (but not in queue or path)
                        visited_only = [n for n in step_data['visited']
                                    if n not in step_data['queue']
                                    and (not step_data['current_path'] or n not in step_data['current_path'])
                                    and n != self.env.start
                                    and n != self.env.end
                                    and n != step_data['expanded_node']]
                        if visited_only:
                            nx.draw_networkx_nodes(G, pos, nodelist=visited_only,
                                                node_color=colors['visited'], node_size=node_size)

                        # 3. Queue nodes (exclude those in path and special nodes)
                        queue_only = [n for n in step_data['queue']
                                    if (not step_data['current_path'] or n not in step_data['current_path'])
                                    and n != self.env.start
                                    and n != self.env.end
                                    and n != step_data['expanded_node']]
                        if queue_only:
                            nx.draw_networkx_nodes(G, pos, nodelist=queue_only,
                                                node_color=colors['queue'], node_size=node_size)

                        # 4. Current path (excluding special nodes)
                        if step_data['current_path']:
                            path_only = [n for n in step_data['current_path']
                                        if n != self.env.start
                                        and n != self.env.end
                                        and n != step_data['expanded_node']]
                            if path_only:
                                nx.draw_networkx_nodes(G, pos, nodelist=path_only,
                                                    node_color=colors['path'], node_size=node_size)

                            # Path edges
                            path_edges = list(zip(step_data['current_path'][:-1], step_data['current_path'][1:]))
                            nx.draw_networkx_edges(G, pos, edgelist=path_edges,
                                                width=path_edge_width, edge_color=colors['path'])

                        # 5. Special nodes - always drawn last to ensure visibility
                        # Current node being expanded
                        nx.draw_networkx_nodes(G, pos, nodelist=[step_data['expanded_node']],
                                            node_color=colors['expanded'], node_size=node_size)

                        # Start and goal
                        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.start],
                                            node_color=colors['start'], node_size=node_size)
                        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.end],
                                            node_color=colors['end'], node_size=node_size)

                        # Draw all edges
                        nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5)

                        # Add labels
                        nx.draw_networkx_labels(G, pos, labels={node: f"{node[0]},{node[1]}" for node in G.nodes()})

                        # Title
                        plt.title(f"BFS Graph Traversal - Step {step_data['step']} / {len(self.steps_data)}")
                        plt.axis('off')

                        # Save frame
                        frame_file = os.path.join(temp_dir, f"frame_{i:03d}.png")
                        plt.savefig(frame_file)
                        frame_files.append(frame_file)
                        plt.close()
                    except Exception as e:
                        plt.close()
                        print(f"Error generating frame {i}: {e}")

                # Generate final frame only if search was successful
                if self.result.success and self.result.path:
                    try:
                        plt.figure(figsize=(size, size))

                        # Use same drawing approach as steps for consistency
                        # 1. Regular nodes (all nodes as background)
                        nx.draw_networkx_nodes(G, pos, node_color=colors['regular'], node_size=node_size)

                        # 2. Visited nodes (that aren't in the final path)
                        visited_only = [n for n in self.result.visited
                                    if n not in self.result.path
                                    and n != self.env.start
                                    and n != self.env.end]
                        if visited_only:
                            nx.draw_networkx_nodes(G, pos, nodelist=visited_only,
                                                node_color=colors['visited'], node_size=node_size)

                        # 3. Path nodes (excluding start/end)
                        path_only = [n for n in self.result.path
                                    if n != self.env.start
                                    and n != self.env.end]
                        if path_only:
                            nx.draw_networkx_nodes(G, pos, nodelist=path_only,
                                                node_color=colors['path'], node_size=node_size)

                        # 4. Special nodes
                        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.start],
                                            node_color=colors['start'], node_size=node_size)
                        nx.draw_networkx_nodes(G, pos, nodelist=[self.env.end],
                                            node_color=colors['end'], node_size=node_size)

                        # Draw all edges
                        nx.draw_networkx_edges(G, pos, width=edge_width, alpha=0.5)

                        # Highlight final path edges
                        path_edges = list(zip(self.result.path[:-1], self.result.path[1:]))
                        nx.draw_networkx_edges(G, pos, edgelist=path_edges,
                                            width=path_edge_width, edge_color=colors['path'])

                        # Add labels
                        nx.draw_networkx_labels(G, pos, labels={node: f"{node[0]},{node[1]}" for node in G.nodes()})

                        plt.title(f"BFS Graph Traversal - Final Path ({len(self.result.path)-1} steps)")
                        plt.axis('off')

                        # Save frame
                        frame_file = os.path.join(temp_dir, "frame_final.png")
                        plt.savefig(frame_file)
                        frame_files.append(frame_file)
                        plt.close()
                    except Exception as e:
                        plt.close()
                        print(f"Error generating final frame: {e}")

                # Create GIF
                try:
                    with imageio.get_writer(output_file, mode='I', fps=fps) as writer:
                        for frame_file in frame_files:
                            image = imageio.imread(frame_file)
                            writer.append_data(image)
                    print(f"Graph animation saved to {output_file}")
                except Exception as e:
                    print(f"Error creating GIF: {e}")

        except Exception as e:
            print(f"Error in BFS graph animation: {e}")


    def create_maze_graph(self):
        """Create a NetworkX graph from maze data.

        Converts the maze grid representation to a graph representation where
        each non-wall cell is a node and edges connect adjacent non-wall cells.

        Returns:
            NetworkX graph representation of the maze.
        """
        G = nx.Graph()

        # Add nodes for all valid positions
        for r in range(self.env.grid.shape[0]):
            for c in range(self.env.grid.shape[1]):
                if self.env.grid[r, c] == 0:  # If not a wall
                    G.add_node((r, c))

        # Add edges between adjacent nodes
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right
        for node in G.nodes():
            r, c = node
            for dr, dc in directions:
                neighbor = (r + dr, c + dc)
                if neighbor in G:  # If neighbor exists in graph
                    G.add_edge(node, neighbor)

        return G

    def _extract_history_data(self):
        """Process exploration history into data for visualization.

        Extracts and processes the exploration history from the search results
        into a format suitable for visualization and educational display.
        """
        self.steps_data = []

        for i, state in enumerate(self.result.exploration_history):
            visited, frontier, current_path, step_info = state

            # Handle case where queue_before might not exist in older data format
            queue_before = step_info.get('queue_before', [])

            self.steps_data.append({
                'step': step_info['step'],
                'expanded_node': step_info['expanded_node'],
                'neighbors_added': step_info['neighbors_added'],
                'queue_before': queue_before,
                'queue_after': step_info['queue_after'],
                'queue': step_info['queue_after'],
                'visited': list(visited),
                'current_path': current_path,
                'visited_count': len(visited),
                'queue_size': len(frontier)
            })

    def visualize_step(self, step_idx: int):
        """Display a single step of the BFS algorithm with educational information.

        Creates a detailed visualization of a specific step in the BFS algorithm's
        execution, showing both the maze state and algorithm state with explanations.

        Args:
            step_idx: Index of the step to visualize.
        """
        if not self.steps_data or step_idx >= len(self.steps_data):
            print("Invalid step index")
            return

        # Get data for the current step
        step_data = self.steps_data[step_idx]

        # Create figure with two subplots
        fig = plt.figure(figsize=(18, 8))

        # Maze visualization on the left
        ax1 = plt.subplot2grid((1, 2), (0, 0))
        self._plot_maze(ax1, step_data)

        # Algorithm state visualization on the right
        ax2 = plt.subplot2grid((1, 2), (0, 1))
        self._plot_bfs_state(ax2, step_data)

        plt.tight_layout()
        plt.show()

        # Print educational explanations
        self._print_step_explanation(step_data)

    def _plot_maze(self, ax, step_data):
        """Plot the maze with current path and visited nodes.

        Creates a visualization of the maze at a specific step in the BFS algorithm,
        color-coding different elements like visited nodes, frontier, and current path.

        Args:
            ax: Matplotlib axis to plot on.
            step_data: Data for the current step.
        """
        # This creates a custom visualization of the maze for the specific step
        # For simplicity, we'll use the environment's visualization function
        # In a full implementation, you'd want to customize this further

        # Create a visualization grid
        rows, cols = self.env.grid.shape
        viz_grid = np.ones((rows, cols)) * 5  # Initialize with unvisited path value

        # Fill in walls
        viz_grid[self.env.grid == 1] = 0  # Walls

        # Fill in visited paths
        for pos in step_data['visited']:
            r, c = pos
            if self.env.grid[r, c] == 0:  # Only if it's a path
                viz_grid[r, c] = 1  # Visited paths

        # Fill in current frontier (queue)
        for pos in step_data['queue']:
            r, c = pos
            if pos != self.env.start and pos != self.env.end:
                viz_grid[r, c] = 6  # Frontier/queue nodes

        # Fill in expanded node
        if step_data['expanded_node'] != self.env.start and step_data['expanded_node'] != self.env.end:
            r, c = step_data['expanded_node']
            viz_grid[r, c] = 7  # Current expanded node

        # Fill in final path
        if step_data['current_path']:
            for pos in step_data['current_path']:
                if pos != self.env.start and pos != self.env.end:
                    r, c = pos
                    viz_grid[r, c] = 2  # Current path

        # Mark start and end
        viz_grid[self.env.start] = 3  # Start
        viz_grid[self.env.end] = 4    # Goal

        # Define colors with enhanced palette
        cmap = plt.cm.colors.ListedColormap([
            'black',     # 0: Wall
            'yellow',    # 1: Visited
            'green',     # 2: Current path
            'blue',      # 3: Start
            'purple',    # 4: Goal
            'white',     # 5: Unvisited path
            'lightblue', # 6: Frontier/queue
            'red'        # 7: Currently expanded node
        ])
        bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5]
        norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

        # Plot the maze
        ax.imshow(viz_grid, cmap=cmap, norm=norm)
        ax.set_title(f"BFS Search - Step {step_data['step']}")
        ax.set_xticks([])
        ax.set_yticks([])

        # Add legend
        legend_elements = [
            plt.Rectangle((0,0), 1, 1, color='white', label='Path'),
            plt.Rectangle((0,0), 1, 1, color='yellow', label='Visited'),
            plt.Rectangle((0,0), 1, 1, color='lightblue', label='Queue (Frontier)'),
            plt.Rectangle((0,0), 1, 1, color='red', label='Current Node'),
            plt.Rectangle((0,0), 1, 1, color='green', label='Current Path'),
            plt.Rectangle((0,0), 1, 1, color='blue', label='Start'),
            plt.Rectangle((0,0), 1, 1, color='purple', label='Goal'),
            plt.Rectangle((0,0), 1, 1, color='black', label='Wall')
        ]
        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3)

    def _plot_bfs_state(self, ax, step_data):
        """Plot BFS algorithm state visualization with queue states.

        Creates a detailed visualization of the BFS algorithm's internal state
        at a specific step, showing metrics, queue state before and after expansion,
        and operation explanations.

        Args:
            ax: Matplotlib axis to plot on.
            step_data: Data for the current step.
        """
        ax.axis('off')

        print(f"step data: {step_data}")

        # Format node coordinates nicely
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Create table with algorithm state information
        table_data = [
            ["Step", str(step_data['step'])],
            ["Expanded Node", format_node(step_data['expanded_node'])],
            ["Visited Nodes", str(step_data['visited_count'])],
            ["Path Length", str(len(step_data['current_path']) if step_data['current_path'] else 0)]
        ]

        # Format neighbors discovered
        if step_data['neighbors_added']:
            neighbors_text = ", ".join([format_node(n) for n in step_data['neighbors_added']])
        else:
            neighbors_text = "None (all neighbors already visited)"
        table_data.append(["Neighbors Added", neighbors_text])

        # Draw the main metrics table
        main_table = ax.table(
            cellText=table_data,
            colLabels=["Metric", "Value"],
            colWidths=[0.3, 0.7],
            loc='center',
            cellLoc='center',
            bbox=[0.1, 0.65, 0.8, 0.3]
        )
        main_table.auto_set_font_size(False)
        main_table.set_fontsize(12)

        # Add styling to the table
        for i in range(len(table_data) + 1):  # +1 for header
            for j in range(2):
                cell = main_table[i, j]
                cell.set_edgecolor('black')
                if i == 0:  # Header
                    cell.set_facecolor('#4472C4')
                    cell.set_text_props(color='white', fontweight='bold')
                else:
                    cell.set_facecolor('#D9E1F2' if i % 2 else '#E9EDF4')

        # Queue BEFORE visualization title
        ax.text(0.3, 0.5, "Queue BEFORE", ha='center', va='center', fontsize=14, fontweight='bold')

        # Queue BEFORE operation
        queue_before_data = []
        # Handle both cases - either queue_before exists or fallback to empty list
        queue_before = step_data.get('queue_before', [])
        for i, node in enumerate(queue_before):
            queue_before_data.append([f"Queue[{i}]", format_node(node)])

        if queue_before_data:
            queue_before_table = ax.table(
                cellText=queue_before_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.05, 0.2, 0.4, 0.25]
            )
            queue_before_table.auto_set_font_size(False)
            queue_before_table.set_fontsize(12)

            # Style queue table
            for i in range(len(queue_before_data) + 1):  # +1 for header
                for j in range(2):
                    cell = queue_before_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#ED7D31')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#FBE5D6' if i % 2 else '#FDF2EA')
        else:
            ax.text(0.3, 0.3, "Queue was empty", ha='center', va='center', fontsize=12, color='red')

        # Queue AFTER visualization title
        ax.text(0.7, 0.5, "Queue AFTER", ha='center', va='center', fontsize=14, fontweight='bold')

        # Queue AFTER operation
        queue_after_data = []
        # Use queue_after if available, otherwise use queue for backward compatibility
        queue_after = step_data.get('queue_after', step_data['queue'])
        for i, node in enumerate(queue_after):
            queue_after_data.append([f"Queue[{i}]", format_node(node)])

        if queue_after_data:
            queue_after_table = ax.table(
                cellText=queue_after_data,
                colLabels=["Index", "Node"],
                colWidths=[0.15, 0.25],
                loc='center',
                cellLoc='center',
                bbox=[0.55, 0.2, 0.4, 0.25]
            )
            queue_after_table.auto_set_font_size(False)
            queue_after_table.set_fontsize(12)

            # Style queue table
            for i in range(len(queue_after_data) + 1):  # +1 for header
                for j in range(2):
                    cell = queue_after_table[i, j]
                    cell.set_edgecolor('black')
                    if i == 0:  # Header
                        cell.set_facecolor('#70AD47')
                        cell.set_text_props(color='white', fontweight='bold')
                    else:
                        cell.set_facecolor('#E2EFDA' if i % 2 else '#EAF5E0')
        else:
            ax.text(0.7, 0.3, "Queue is now empty", ha='center', va='center', fontsize=12, color='red')

        # Add operation explanation
        operation_explanation = [
            f"1. Dequeued {format_node(step_data['expanded_node'])} from front of queue",
            f"2. Checked if it's the goal node",
            f"3. Examined unvisited neighbors"
        ]
        if step_data['neighbors_added']:
            neighbors = ", ".join([format_node(n) for n in step_data['neighbors_added']])
            operation_explanation.append(f"4. Added neighbors to queue: {neighbors}")
        else:
            operation_explanation.append("4. No new neighbors to add")

        # Draw operation explanation box
        ax.text(0.5, 0.1, "\n".join(operation_explanation), ha='center', va='center',
                fontsize=10, bbox=dict(boxstyle="round,pad=0.5", facecolor='#FFFFCC', alpha=0.5))

    def create_gif(self, filename='bfs_animation.gif', fps=1, dpi=100):
        """Create a GIF animation from BFS steps.

        Generates an animated GIF showing the progression of the BFS algorithm
        step by step, including both maze state and algorithm state.

        Args:
            filename: Path to save the output GIF.
            fps: Frames per second for the animation.
            dpi: Resolution (dots per inch) for the output.
        """
        print(f"Creating GIF with {len(self.steps_data)} frames...")

        # Create temporary directory for frames
        with tempfile.TemporaryDirectory() as temp_dir:
            # Save frames
            frame_files = []
            for i, step_data in enumerate(self.steps_data):
                # Create figure
                fig = plt.figure(figsize=(18, 8))

                # Maze visualization
                ax1 = plt.subplot2grid((1, 2), (0, 0))
                self._plot_maze(ax1, step_data)

                # Algorithm state visualization
                ax2 = plt.subplot2grid((1, 2), (0, 1))
                self._plot_bfs_state(ax2, step_data)

                plt.tight_layout()

                # Save frame
                frame_file = os.path.join(temp_dir, f"frame_{i:03d}.png")
                plt.savefig(frame_file, dpi=dpi)
                frame_files.append(frame_file)
                plt.close(fig)

            # Create GIF
            with imageio.get_writer(filename, mode='I', fps=fps) as writer:
                for frame_file in frame_files:
                    image = imageio.imread(frame_file)
                    writer.append_data(image)

        print(f"GIF animation saved to {filename}")

    def _print_step_explanation(self, step_data):
        """Print educational explanation about the current BFS step.

        Provides detailed educational commentary about what happens during
        a specific step of the BFS algorithm, including process explanations
        and algorithm properties.

        Args:
            step_data: Data for the current step.
        """
        # Format node coordinates nicely
        def format_node(node):
            return f"({node[0]},{node[1]})"

        # Educational commentary
        expanded_node = format_node(step_data['expanded_node'])
        print(f"🔍 BFS Step {step_data['step']} Explanation:")
        print(f"---------------------------")
        print(f"Currently expanding: {expanded_node}")

        # Explain BFS processing for this step
        print("\n1️⃣ BFS Algorithm Process:")
        print(f"   - Dequeued node {expanded_node} from the front of the queue")
        print(f"   - Checked if it's the goal (it's {'the goal! 🎉' if step_data['expanded_node'] == self.env.end else 'not the goal'})")

        if step_data['expanded_node'] != self.env.end:
            # Explain how neighbors are processed
            if step_data['neighbors_added']:
                neighbors_text = ", ".join([format_node(n) for n in step_data['neighbors_added']])
                print(f"\n2️⃣ Discovered {len(step_data['neighbors_added'])} unvisited neighbors: {neighbors_text}")
                print("   These neighbors were added to the back of the queue for later exploration.")
                print("   💡 This is why BFS explores nodes in order of their distance from the start.")
            else:
                print("\n2️⃣ No new neighbors were discovered (all neighbors are already visited)")

        # Explain queue state
        print(f"\n3️⃣ Queue status:")
        if step_data['queue']:
            queue_text = ", ".join([format_node(n) for n in step_data['queue']])
            print(f"   Queue now contains: {queue_text}")
            print(f"   Next node to explore will be: {format_node(step_data['queue'][0])}")
        else:
            print("   Queue is now empty. Search will terminate.")

        # Relate to BFS properties
        print("\n4️⃣ BFS Properties:")
        print("   - BFS always finds the shortest path in unweighted graphs")
        print("   - Time complexity: O(V + E) where V is vertices and E is edges")
        print("   - Space complexity: O(V) for the queue and visited set")

        # Current statistics
        print(f"\n📊 Current Statistics:")
        print(f"   - Visited {step_data['visited_count']} nodes so far")
        print(f"   - Queue size: {step_data['queue_size']}")
        print(f"   - Current path length: {len(step_data['current_path']) if step_data['current_path'] else 0}")



## Reports

In [ ]:
# Example of using the enhanced BFS visualization tools
from typing import Tuple, List, Dict, Optional, Union, Callable, Set, Any
from dataclasses import dataclass, field, asdict
from collections import deque


# Create a maze with the educational BFS implementation
config = Config(maze_size=5, show_exploration=True, visualization_delay=0.8)
env = MazeEnvironment(config)

# Create enhanced BFS instance
bfs = BreadthFirstSearch(env)

# Run the search and collect educational results
result = bfs.run()

# Option 1: Use the built-in enhanced visualization
print("Running enhanced BFS visualization...")
bfs.visualize_search(result)

# Option 2: Create an educational dashboard
dashboard = BFSEducationalDashboard(env, result)

# Show how to use the dashboard
print("\n\nCreating BFS Educational Dashboard")
print("-----------------------------------")

# Visualize a specific step
print("\nShowing step 3 as an example:")
dashboard.visualize_step(3)

# Analyze BFS performance
print("\n\nBFS Performance Analysis")
print("------------------------")
print(f"Search success: {result.success}")
print(f"Total steps: {result.steps}")
print(f"Visited nodes: {len(result.visited)}")
if result.path:
    print(f"Path length: {len(result.path)}")
    print(f"Path efficiency: {len(result.path)/len(result.visited):.2f} (path length / visited nodes)")


In [ ]:
dashboard.create_gif(fps=1.5)

In [ ]:
dashboard = BFSEducationalDashboard(env, result)
dashboard.animate_bfs_on_graph(fps=3, size=6)  # Animated view